In [ ]:
import sys
import tensorflow as tf
import numpy as np
sys.path.append("../") 


In [ ]:
from src.preprocessing import load_and_split_data, create_generators
from src.postprocessing import evaluate_model

2025-05-25 19:13:19.137723: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/ayushgupta/Desktop/Annam/soil_classificaton_annam/challenge-1/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
label_path = "../data/dataset/soil_classification-2025/train_labels.csv"
train_path = "../data/dataset/soil_classification-2025/train"

In [ ]:
train_df, valid_df = load_and_split_data(label_path)
train_generator, validation_generator = create_generators(train_df, valid_df, train_path)

In [ ]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(331,331,3)
                     )

base_model.trainable=False
input_tensor = tf.keras.Input(shape=(331,331,3))
output_tensor = base_model(input_tensor)

In [ ]:
model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.Dense(512, activation='relu' ,kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
early = tf.keras.callbacks.EarlyStopping( patience=10,
                                          min_delta=0.001,
                                          restore_best_weights=True)

In [ ]:
batch_size=32
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = validation_generator.n//validation_generator.batch_size
history = model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,
                    callbacks=[early])

In [ ]:
model.save("soil_model.h5")

In [ ]:
evaluate_model(model, validation_generator, output_json_path="../metrics.json")